<a href="https://colab.research.google.com/github/alvarezhaizea/03MIAR---Algoritmo-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Haizea_Alvarez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Haizea Alvarez Merino  <br>
Url: https://github.com/alvarezhaizea/03MIAR---Algoritmo-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Haizea_Alvarez.ipynb <br>
Google Colab: https://colab.research.google.com/drive/1MRJfQgLxvc4_dQdtgZPbuT6T-FqAHngb <br>

**Problema:**
>1. Sesiones de doblaje <br>

**Descripción del problema:**

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.







                                        

#Modelo

 **¿Como represento el espacio de soluciones?**

He utilizado el algoritmo genético para resolver el problema de asignación de tomas a los días de doblaje. El espacio de soluciones se representa como una población de individuos, donde cada individuo es una lista que está compuesto por diferentes sublistas, en el que cada una de las listas representan un día de dobalje y los valores dentro de estas sublistas las tomas asignadas a ese día.

**¿Cual es la función objetivo?**

La función objetivo a minimizar es el coste total del doblaje por los servicios de los actos. El coste se define como distancia en el código dentro de la función *evaluar_poblacion*.

En este caso, la distancia representa la cantidad de días en los que un actor debe grabar, penalizando aquellas soluciones en las que los actores trabajan en demasiados días.


Por lo tanto, la función objetivo se define como:

$$
\min d = \sum_{i=1}^{n} d_i X_i
$$

Donde:  
- $d$ es la distancia total a minimizar.  
- $d_i$ es el coste total del actor $i$.  
- $X_i$ hace referencia al actor $i$.  

**¿Como implemento las restricciones?**

Las restricciones que hemos tenido en cuenta son las siguientes:
1. Cada toma aparece solo una vez por individuo y todas las tomas se tienen que grabar durante el rodaje.
2. Cada día tiene máximo 6 tomas.

Por un lado, la primera restricción se controla durante la generación de los individuos mediante el cruce, y se realiza un segundo control al asignar las distancias a cada individuo. De esta forma, los individuos que no cumplan con esta restricción tendrán un mayor peso.

Por otro lado, la segunda restricción se controla desde el inicio, ya que la población inicial se crea teniendo en cuenta esta restricción y la longitud de las sublistas no se modifica en ningún momento.

#Análisis

**¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones**

La complejidad del problema es difícil de cuantificar. Pero, si que conocemos la complejidad del algoritmo de fuerza bruta, que es $O(n!)$, ya que debe explorar todas las posibles soluciones.

Para calcular el número de soluciones, sabemos que tenemos 30 tomas y que cada día se pueden grabar un máximo de 6 tomas. Por lo tanto, supongamos que se requieren 5 días para completar el doblaje. Es importante destacar que el orden de las tomas dentro de cada día no importa y que no se puede elegir la misma toma más de una vez.

Por lo tanto, la fórmula que tendría que usar sería:

$$
C(n+k-1, k-1) = C(30+5-1, 5-1) = C(34, 4)
$$

Esta fórmula se conoce como combinaciones con repetición y calcula el número total de formas de distribuir las 30 tomas en los 5 días.

Y el resultado sería:

$$
C(34, 4) = \frac{34!}{4!(34-4)!} = 46376
$$

#Diseño

**¿Que técnica utilizo? ¿Por qué?**

El algoritmo genético se ha elegido para resolver este problema debido a varias razones clave:

1. **Adaptabilidad a problemas NP-hard:** Los algoritmos genéticos dan muy buenos resultados en problemas de combinatoria complejos como este, donde la búsqueda en el espacio de soluciones es costosa.

2. **Flexibilidad en la modelización:** Los AG se adaptan perfectamente al problema, representando tomas, días de rodaje y la planificación del rodaje como genes, cromosomas e individuos, respectivamente.

3. **Escapatoria de óptimos locales:** Las operaciones de mutación y cruce permiten escapar de los óptimos locales, generando soluciones diversas que cumplen con las restricciones.

4. **Escalabilidad y eficiencia:** Son fácilmente escalables, permitiendo ajustar parámetros como tamaño de población o generaciones según los recursos disponibles o el balance entre calidad y tiempo de cálculo.

In [42]:
import random
from google.colab import drive
import pandas as pd
from tabulate import tabulate


De la siguiente manera cargamos los datos. No obstante, si se ejecuta desde otra cuenta, podría ocurrir un error porque los archivos están en una carpeta de mi Drive. Por eso, he optado por almacenarlos en una variable y he dejado comentada la forma en la que cargué los datos, para evitar posibles errores.

In [43]:
# drive.mount('/content/drive')
# ruta_csv = "/content/drive/My Drive/Colab Notebooks/Trabajo AO/Problema1.csv"

# df = pd.read_csv(ruta_csv, header=None)
# df = df.iloc[2:32, 1:11]
# df = df.astype(int)


# # Convertir a lista de listas
# tomas_disponibilidad = df.values.tolist()

# for fila in tomas_disponibilidad:
#     print(fila)

In [44]:
tomas_disponibilidad = [[1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                        [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                        [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
                        [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
                        [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
                        [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                        [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                        [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
                        [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
                        [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
                        [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
                        [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
                        [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                        [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                        [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
                        [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                        [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                        [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
                        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                        [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                        [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
                        [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
                        [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                        [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                        [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
                        [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]]


In [45]:
def verificar_completitud(individuo):
  '''
  Comprobamos que todas las tomas aparecen. Que esto se cumpla es equivalente a mirar que no hay tomas repetidas.
  Esto se debe a que como el tamaño de los individuos no es modificable si están todos los numetos es que no hay
  repeticiones ya que sino faltaria una toma.
  '''
  todos_los_numeros = [toma for dia in individuo for toma in dia]

  if set(todos_los_numeros) == set(range(30)):
    return True  # Todos los números están presentes
  else:
    return False  # Faltan algunos números

In [46]:
def generar_poblacion(lista_tomas, N):
  poblacion = []
  for _ in range(N):
    solucion = []
    random.shuffle(lista_tomas)  # Mezclar las tomas

    dia_actual = []
    for toma in lista_tomas:
      if len(dia_actual) < 6:  # Si el día actual tiene menos de 6 tomas, lo añadimos a la lista dia_actual
          dia_actual.append(toma)
      else:  # Si el día tiene 6 tomas, asignar un nuevo día
          solucion.append(dia_actual)
          dia_actual = [toma]

    if len(dia_actual) > 0: #En el caso de que el numero de tomas no sea multiplo de 6 tendremos un día "incompleto". En ese caso formazos la asignacion del dia
      solucion.append(dia_actual)

    poblacion.append(solucion)
  return poblacion

In [47]:
def evaluar_poblacion(poblacion, data):
  mejor_solucion = None
  mejor_distancia = float('inf')

  for individuo in poblacion:
    distancia = 0
    tomas_vistas = set()

    # Inicializamos un diccionario para contar los días de grabación por actor
    dias_grabacion_por_actor = {actor_idx: set() for actor_idx in range(len(data[0]))}

    # Evaluamos las tomas
    for dia_idx, dia in enumerate(individuo):
      # Ahora verificamos la disponibilidad de los actores para las tomas de este día
      for toma in dia:
          for actor_idx in range(len(data[0])):
              if data[toma][actor_idx] == 1:  # Si el actor está disponible para la toma
                  dias_grabacion_por_actor[actor_idx].add(dia_idx)  # Se agrega el día de grabación para el actor

    # Penalizamos si algún actor graba en muchos días
    for actor_idx, dias in dias_grabacion_por_actor.items():
      dias_grabacion = len(dias)
      # Calcular penalización proporcional a la cantidad de días de grabación
      # Penalizamos más si graba más días.
      penalizacion = max(0, dias_grabacion - 1)
      distancia += penalizacion

    # Comparamos y guardamos la mejor solución
    if distancia < mejor_distancia:
      mejor_solucion = individuo
      mejor_distancia = distancia

  return mejor_solucion, mejor_distancia

In [48]:
def seleccion_padres(poblacion, data):
  poblacion_ordenada = sorted(poblacion, key=lambda x: evaluar_poblacion([x], data)[1])
  padres = poblacion_ordenada[:2] #Nos quedamos con los dos mejores individuos
  return padres[0], padres[1]

In [49]:
def Cruzar(poblacion, data, N):
  nueva_poblacion = []
  i = 0
  # Mientras la nueva población no tenga el mismo tamaño que la población original
  while i < N:
    padre1, padre2 = seleccion_padres(poblacion, data)
    # Seleccionar dos padres aleatorios de la población actual
    padre1 = random.choice(poblacion)
    padre2 = random.choice(poblacion)

    # Realizamos un cruce entre dos soluciones asegurando que no haya repeticiones de tomas
    punto_cruce = random.randint(1, min(len(padre1), len(padre2)) - 1)

    # Crear los hijos a partir del cruce
    hijo1 = padre1[:punto_cruce] + padre2[punto_cruce:]
    hijo2 = padre2[:punto_cruce] + padre1[punto_cruce:]

    if verificar_completitud(hijo1):
      nueva_poblacion.append(hijo1)

    if verificar_completitud(hijo2):
      nueva_poblacion.append(hijo2)
    i += 1
  nueva_poblacion = nueva_poblacion + poblacion


  return nueva_poblacion

In [50]:
def Mutacion(poblacion, probabilidad_mutacion=0.15):
  nueva_poblacion = []

  for individuo in poblacion:
    if random.random() < probabilidad_mutacion:  # Aplicar mutación con la probabilidad indicada
      hijo = [dia.copy() for dia in individuo]  # Copiar el individuo para no modificar el original

      # Elegir dos días aleatorios
      dia_origen = random.choice(hijo)
      dia_destino = random.choice(hijo)

      # Elegir dos tomas aleatorias, una del día de origen y otra del día de destino
      toma_origen = random.choice(dia_origen)
      toma_destino = random.choice(dia_destino)

      # Evitar que las tomas sean la misma, es decir, la misma posicion, por eso hacemos el cambio solo si son diferentes
      if toma_origen != toma_destino:
        # Intercambiar las tomas entre los dos días
        dia_origen.remove(toma_origen)
        dia_destino.remove(toma_destino)
        dia_origen.append(toma_destino)
        dia_destino.append(toma_origen)
        nueva_poblacion.append(hijo)
  nueva_poblacion = nueva_poblacion + poblacion
  return nueva_poblacion

In [51]:
def Seleccionar(data, poblacion, N, elitismo):
  poblacion_ordenada = sorted(poblacion, key=lambda x: evaluar_poblacion([x], data)[1])
  elite_size = int(elitismo * N)
  #Seleccionamos los mejores X individuos y los demás se seleccionar aleatoriamente.
  nueva_poblacion = poblacion_ordenada[:elite_size]
  while len(nueva_poblacion) < N:
    individuo = random.choice(poblacion_ordenada[elite_size:])
    poblacion_ordenada.remove(individuo)
    nueva_poblacion.append(individuo)
  return nueva_poblacion

In [52]:
def algoritmo_genetico(data, N=100, elitismo=0.1, mutacion=0.15, generaciones=100):
  Tomas = list(range(len(data)))
  poblacion = generar_poblacion(Tomas, N)
  mejor_solucion, mejor_distancia = evaluar_poblacion(poblacion, data)

  n = 1
  while n < generaciones:
    nueva_poblacion = Cruzar(poblacion, data, N)

    nueva_poblacion = Mutacion(nueva_poblacion, probabilidad_mutacion=mutacion)

    # Selección de N individuos
    poblacion = Seleccionar(data, nueva_poblacion, N, elitismo)

    mejor_solucion, mejor_distancia = evaluar_poblacion(poblacion, data)

    print(f"Generación #{n}\nMejor solución: {mejor_solucion}\nCon distancia: {mejor_distancia}\n")

    n += 1

  return mejor_solucion, mejor_distancia

In [53]:
# Ejecutar el algoritmo genético con los datos
mejor_solucion, mejor_distancia = algoritmo_genetico(tomas_disponibilidad, N=50, elitismo=0.1, mutacion=0.15, generaciones=50)

Generación #1
Mejor solución: [[9, 14, 15, 29, 17, 24], [1, 19, 22, 18, 8, 5], [11, 7, 10, 4, 3, 0], [23, 20, 13, 12, 28, 26], [21, 25, 2, 16, 6, 27]]
Con distancia: 24

Generación #2
Mejor solución: [[9, 14, 15, 29, 17, 24], [1, 19, 22, 18, 8, 5], [11, 7, 10, 4, 3, 0], [23, 20, 13, 12, 28, 26], [21, 25, 2, 16, 6, 27]]
Con distancia: 24

Generación #3
Mejor solución: [[9, 14, 15, 29, 17, 24], [1, 19, 22, 18, 8, 5], [11, 7, 10, 4, 3, 0], [23, 20, 13, 12, 28, 26], [21, 25, 2, 16, 6, 27]]
Con distancia: 24

Generación #4
Mejor solución: [[9, 14, 15, 29, 17, 24], [1, 19, 22, 18, 8, 5], [11, 7, 10, 4, 3, 0], [23, 20, 13, 12, 28, 26], [21, 25, 2, 16, 6, 27]]
Con distancia: 24

Generación #5
Mejor solución: [[9, 14, 15, 29, 17, 24], [1, 19, 22, 18, 8, 5], [11, 7, 10, 4, 3, 0], [23, 20, 13, 12, 28, 26], [21, 25, 2, 16, 6, 27]]
Con distancia: 24

Generación #6
Mejor solución: [[9, 14, 15, 29, 17, 24], [1, 19, 22, 18, 8, 5], [11, 7, 10, 4, 3, 0], [23, 20, 13, 12, 28, 26], [21, 25, 2, 16, 6, 27]]

In [54]:
dias = [f"Día {i+1}" for i in range(len(mejor_solucion))]
tabla = tabulate(mejor_solucion, headers=["Grabación 1", "Grabación 2", "Grabación 3", "Grabación 4", "Grabación 5", "Grabación 6"], showindex=dias, tablefmt="grid")

# Mostrar tabla
print(tabla)

+-------+---------------+---------------+---------------+---------------+---------------+---------------+
|       |   Grabación 1 |   Grabación 2 |   Grabación 3 |   Grabación 4 |   Grabación 5 |   Grabación 6 |
+=======+===============+===============+===============+===============+===============+===============+
| Día 1 |             9 |            15 |            17 |            24 |            29 |            13 |
+-------+---------------+---------------+---------------+---------------+---------------+---------------+
| Día 2 |             1 |            19 |            22 |            18 |             5 |             6 |
+-------+---------------+---------------+---------------+---------------+---------------+---------------+
| Día 3 |            11 |             7 |            10 |             4 |             0 |            16 |
+-------+---------------+---------------+---------------+---------------+---------------+---------------+
| Día 4 |            23 |            20 |     